In [ ]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd /content/yolov5
!pip install -r requirements.txt  # install dependencies

In [3]:
import torch
from IPython.display import Image  
print('torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

torch 1.13.0+cu116 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [ ]:
!pip install roboflow

In [4]:
from roboflow import Roboflow
rf = Roboflow(api_key="pwB1fz1QMtUQxrO9XFNE")
project = rf.workspace("yolov5-bzr4l").project("yolov5-zg2oy")
dataset = project.version(4).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to yolov5-4 in yolov5pytorch:: 100%|██████████| 6440/6440 [00:02<00:00, 3215.24it/s]


In [ ]:
%cat /content/yolov5/yolov5-1/data.yaml

path: /content/yolov5/yolov5-1

train: train/images
val: valid/images
test: test/images

nc: 2
names: ['car', 'person']

In [7]:
# define number of classes based on YAML
import yaml
with open('/content/yolov5/yolov5-4/data.yaml', 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [8]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [9]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [13]:
# Train YOLOv5s on COCO128 for 3 epochs
! python train.py --img 640 --batch 16 --epochs 120 --data /content/yolov5/yolov5-4/data.yaml --cfg /content/yolov5/models/custom_yolov5s.yaml --weights ' '

train: weights= , cfg=/content/yolov5/models/custom_yolov5s.yaml, data=/content/yolov5/yolov5-4/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=120, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-47-g2370a55 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_

In [14]:
!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 640 --conf 0.2 --source /content/yolov5/yolov5-4/test/images

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/yolov5/yolov5-4/test/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.2, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-47-g2370a55 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7251912 parameters, 0 gradients
image 1/106 /content/yolov5/yolov5-4/test/images/13_jpg.rf.0c7213b53bb5ba722810b9af950cb7a6.jpg: 640x640 1 car, 13.7ms
image 2/106 /content/yolov5/yolov5-4/test/images/15_jpg.rf.d208edc9bf576c876c0ec6792353f8c0.jpg: 640x640 3 cars, 13.7ms
image 3/106 /content/yolov5/yolov5-4/test/images/18_jpg.rf.7ea05891a5c24a681a46c

In [16]:
import torch

model = torch.hub.load('.', 'custom', '/content/yolov5/runs/train/exp/weights/best.pt', source='local')

YOLOv5 🚀 v7.0-47-g2370a55 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
custom_YOLOv5s summary: 182 layers, 7251912 parameters, 0 gradients
Adding AutoShape... 


In [86]:
import cv2
import numpy as np
from scipy.spatial import distance

In [68]:
# Inference
imgs=cv2.imread('/content/Screenshot 2022-12-24 at 10.13.19 PM.png')
results = model(imgs)

# Results
results.print()
results.save()  # or .show()

results.xyxy[0]  # img1 predictions (tensor)
bb_df=results.pandas().xyxy[0] 

image 1/1: 822x1240 2 persons
Speed: 2.9ms pre-process, 10.9ms inference, 1.4ms NMS per image at shape (1, 3, 448, 640)
Saved 1 image to runs/detect/exp13


In [74]:
bb_df['x_center']=bb_df['xmax']-bb_df['xmin']
bb_df['y_center']=bb_df['ymax']-bb_df['ymin']
bb_df['center']=[np.round(bb_df['x_center'].values,2),np.round(bb_df['y_center'].values,2)]

In [78]:
bb_df

,xmin,ymin,xmax,ymax,confidence,class,name,x_center,y_center,center
0,845.602844,352.533905,1019.722900,656.408997,0.538166,2,person,174.120056,303.875092,"[174.12, 196.53]"
1,800.285583,157.648102,996.817871,628.915222,0.336360,2,person,196.532288,471.267120,"[303.88, 471.27]"


In [103]:
car=[]
person=[]
dista=[]
for i,j in bb_df[['class','center']].values:
  if i==1:
    car.append(j)
  elif i==2:
    person.append(j)
if len(car)==0 or len(person)==0:
  print('Detected bounding boxes are belongs to same class')
else:
  for k in range(len(car)):
    for l in range(len(person)):
      dista.append(distance.euclidean(car[k], person[l]))
  print('mean distance between two classes',int(np.array(dista).mean()))

mean distance between two classes 366
